# Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=561

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import glob
import json
import cv2
import numpy as np
from skimage.feature import graycomatrix
import h5py

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [3]:
def get_path_and_lable(src_path, num_files_per_folder):
    image_paths = []
    lesions = []

    for root, dirs, files in tqdm(os.walk(src_path), desc='Walking directories', unit=' dir'):
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            image_files = sorted(glob.glob(os.path.join(dir_path, '*.jpg')))

            for image_file in image_files[:num_files_per_folder]:
                filename = os.path.basename(image_file)
                parts = filename.split('_')
                if len(parts) >= 3:
                    second_part = parts[2]
                    lesions.append(second_part)
                    image_paths.append(image_file)

    return image_paths, lesions

class ImageFeatureExtractor:
    def __init__(self, target_size):
        self.target_size = target_size

    def preprocess_image(self, image_path):
        image = cv2.imread(image_path)
        image = cv2.resize(image, self.target_size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def extract_color_histogram(self, image):
        histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        histogram = cv2.normalize(histogram, histogram).flatten()
        return histogram

    def extract_hu_moments(self, image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        moments = cv2.HuMoments(cv2.moments(gray_image)).flatten()
        return moments

    def extract_haralick_texture(self, image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        texture = graycomatrix(gray_image, distances=[1], angles=[0], symmetric=True, normed=True)
        haralick_features = np.mean(texture, axis=(0, 1, 2))
        return haralick_features

    def extract_features(self, image_path):
        image = self.preprocess_image(image_path)
        color_histogram = self.extract_color_histogram(image)
        hu_moments = self.extract_hu_moments(image)
        haralick_texture = self.extract_haralick_texture(image)
        global_features = np.concatenate([color_histogram, hu_moments, haralick_texture])
        return global_features

    def save_features_to_hdf5(self, features, lesions, output_path):
        with h5py.File(output_path, 'w') as f:
            f.create_dataset('features', data=features)
            f.create_dataset('lesions', data=lesions)

def load_data_from_hdf5(file_path):
    with h5py.File(file_path, 'r') as f:
        features = f['features'][:]
        lesions = f['lesions'][:]
    return features, lesions

# camera cat

In [6]:
src_path = '/content/drive/Shareddrives/반려묘/일반카메라'
num_files_per_folder = 1000
buffer_size = 1000
batch_size = 64
target_size = (96, 96)

image_paths, lesions = get_path_and_lable(src_path, num_files_per_folder)
lesions = np.array([s.encode('utf-8') for s in lesions])

Walking directories: 15 dir [00:02,  5.29 dir/s]


In [7]:
feature_extractor = ImageFeatureExtractor(target_size=(96, 96))

features = []
for image_path in tqdm(image_paths):
    image_features = feature_extractor.extract_features(image_path)
    features.append(image_features)

features = np.array(features)
lesions = np.array(lesions)

output_path = 'features.h5'
feature_extractor.save_features_to_hdf5(features, lesions, output_path)

100%|██████████| 10675/10675 [04:59<00:00, 35.59it/s]


In [8]:
hdf5_file_path = 'features.h5'

features, lesions = load_data_from_hdf5(hdf5_file_path)

X_train, X_test, y_train, y_test = train_test_split(features, lesions, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [9]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(96, 96, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64,(3 ,3), padding ='same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2 , 2)))

model.add(Dropout(0.25))

model.add(Conv2D(128,(3 ,3), padding ='same', activation ='relu'))
model.add(Conv2D(128,(3 ,3), padding ='same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2 , 2)))

model.add(Dropout (0.25))

model.add(Flatten())

model.add(Dense (1024 ,activation = 'relu' ))
model.add(BatchNormalization ())
Model.add(Dropout (0.5 ))

Model.add(Dense (4 ,activation = 'softmax' ))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/150


ValueError: ignored

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(4):
    fpr[i], tpr[i], _ = roc_curve(y_true_classes, y_pred[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8, 6))
for i in range(4):
    plt.plot(fpr[i], tpr[i], lw=2, label=f'ROC curve (class {i}, AUC = {roc_auc[i:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('Mk-1.tflite', 'wb') as f:
    f.write(tflite_model)

# microscope cat

In [ ]:
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Concatenate, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mean_squared_error

def inception_module(input_layer, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), activation='relu')(input_layer)
    conv3x3_reduce = Conv2D(filters[1], (1, 1), activation='relu')(input_layer)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)
    conv5x5_reduce = Conv2D(filters[3], (1, 1), activation='relu')(input_layer)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_layer)
    maxpool_conv = Conv2D(filters[5], (1, 1), activation='relu')(maxpool)
    inception_output = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool_conv])
    return inception_output

# Input
input_shape = (128, 128, 3)
input_layer = Input(shape=input_shape)

# Inception block
inception_output = inception_module(input_layer, filters=[64, 128, 192, 32, 96, 64])
inception_output = inception_module(inception_output, filters=[64, 128, 192, 32, 96, 64])
# Add more inception modules if needed

# Primary Capsule layer
primary_capsules = Conv2D(32, (1, 1), activation='relu')(inception_output)

# Higher Capsule layers
# (Add imperative routing mechanism layers here)

# PReLU activation for routing
higher_capsules_prelu = PReLU()(higher_capsules)

# Flatten and Fully Connected layers
capsule_flatten = Flatten()(higher_capsules_prelu)  # Flatten higher capsules
output_layer = Dense(2, activation='softmax')(capsule_flatten)  # Two capsules: parasitized and uninfected

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with Adam optimizer and custom loss function
optimizer = Adam(learning_rate=0.007, beta_1=0.8)
loss_fn = custom_loss_function # Define the custom loss function as described in the paper
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()